In [28]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search, Q
import pandas as pd

## Final result example. Relevant data gets prepared, uploaded to elasticsearch, and matched against bank transactions

# Load the dataframes
df_bank = pd.read_csv('csv/01-bank.csv')
df_client = pd.read_csv('csv/02-client.csv')
df_student = pd.read_csv('csv/03-student.csv')

# Merge the clients and students dataframes
df_client_combined = pd.merge(df_client, df_student, left_on='client id', right_on='associated client id')

# Pivot the data to put all student info in the same row as clients
df_client_combined = df_client_combined.pivot_table(
    index=['client id', 'name', 'last name', 'email1', 'email2', 'handle', 'account number'],
    columns=df_client_combined.groupby("client id").cumcount() + 1,
    values=['student name', 'student last name', 'grade'],
    aggfunc='first'
).reset_index()

# Flatten the previous pivot table
df_client_combined.columns = df_client_combined.columns.to_series().apply(
    lambda x: ' '.join(str(y) for y in x if y).strip()
)

# Prepare the bank search terms (search the sender and description values of the transactions)
df_bank['bank search terms'] = df_bank['sender'].fillna('') + ' ' + df_bank['description'].fillna('')

# Function to upload the combined client data to elasticsearch (make the data searchable)
def upload_data_to_elasticsearch():
    index_name = "es_client_combined"

    # Setup Elasticsearch connection
    es = Elasticsearch(
        'https://elastic:9200',
        basic_auth=('elastic', 'password'),
        verify_certs=False,
        ssl_show_warn=False
    )

    try:
        # Check if index exists, delete if it does
        if es.indices.exists(index=index_name):
            es.indices.delete(index=index_name)
        
        # Create a new index
        es.indices.create(index=index_name)

        # Prepare and upload data to elasticsearch
        actions = [
            {
                "_index": index_name,
                "_id": str(record['client id']),
                "_source": record,
            }
            for record in df_client_combined.to_dict(orient='records')
        ]
        # Perform the bulk upload
        helpers.bulk(es, actions)

        # Refresh the index to make the changes searchable
        es.indices.refresh(index=index_name)
    
    except Exception as e:
        pass  # Handle exceptions as needed

# Upload the combined client data to elasticsearch
upload_data_to_elasticsearch()

def get_highest_relevance_clientid(dataframe, index_name, min_score_difference=1.0):
    # Establish the connection to elasticsearch
    es = Elasticsearch(
        'https://elastic:9200',
        basic_auth=('elastic', 'password'),
        verify_certs=False,
        ssl_show_warn=False
    )

    # Get the highest relevance client id for bank search terms
    def get_clientid(text):
        # Clean up the search terms
        text = text.strip()

        # Skip empty search terms
        if not text:
            return None

        # Create a multi_match query # Pending. Boost last name/client id importance, ignore single letter words (false positives)
        query = Q('multi_match', query=text, fields=['*'], type='best_fields', minimum_should_match=1)
        
        # Execute the search
        s = Search(using=es, index=index_name).query(query).extra(size=2)
        response = s.execute()

        ## This is the part that needs to be tweaked/adjusted for edge cases, along with with boosting field importance 
        # Handle ambiguity if there are multiple hits
        if len(response.hits) > 0:
            # If there's only one hit, or the score difference between the top two hits is above the threshold
            if len(response.hits) == 1 or response.hits[0].meta.score - response.hits[1].meta.score >= min_score_difference:
                return response.hits[0].meta.id
        # Return None if the top hits are ambiguous or there are no hits
        return None

    # Search each row in the df_bank "bank search terms" and store the results in a new column "matched client id"
    dataframe['matched client id'] = dataframe['bank search terms'].apply(get_clientid).astype('Int64')

    return dataframe

# Perform the client id matching on the bank dataframe
df_bank_matched = get_highest_relevance_clientid(df_bank, 'es_client_combined')

# Display the updated dataframe with matched client id
df_bank_matched

,unique transaction id,date,transaction type,sender,description,amount,bank search terms,matched client id
0,10000000,24/11/22,direct debit received,Acme Inc.,Lwilkerson,701.16,Acme Inc. Lwilkerson,33153
1,10000001,27/02/22,rejected direct debit,Witch Foods,Globex Corporation,-223.16,Witch Foods Globex Corporation,<NA>
2,10000002,13/01/22,direct debit received,Acme Inc.,Gaven Ariana 4 8,386.13,Acme Inc. Gaven Ariana 4 8,33139
3,10000003,21/05/22,transfer received,Globex Corporation,Jessie and Raymond,732.43,Globex Corporation Jessie and Raymond,33126
4,10000004,06/05/22,Bank fee,NaN,NaN,-272.75,,<NA>
5,10000005,25/04/22,direct debit received,Acme Inc.,Bob,581.58,Acme Inc. Bob,<NA>
6,10000006,13/03/22,transfer received,NaN,33129,856.76,33129,33129
7,10000007,22/02/22,Expense payment,NaN,NaN,-434.58,,<NA>
8,10000008,13/12/22,Bank fee,NaN,NaN,-196.26,,<NA>
9,10000009,07/10/22,transfer received,Initech,Savanna Lugo,376.48,Initech Savanna Lugo,33116
